In [1]:
import asyncio
from autobahn.asyncio.websocket import WebSocketServerProtocol, WebSocketServerFactory
import websockets
import numpy
import threading
import time

from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

In [2]:
loop = asyncio.get_event_loop()

class WebIOThread(threading.Thread):
    def __init__(self):
        super().__init__()
        self.loop = asyncio.get_event_loop()
        self.should_quit = False
    def quit(self):
        self.should_quit = True
        #print(self.loop.is_running())
        if (self.loop.is_running):
            self.loop.call_soon_threadsafe(self.loop.stop)
    
    def run(self):
        self.running = True
        print("thread start")    
        while (not self.should_quit):
        #logging.debug('running')
            print("start loop")
            self.loop.run_forever()
            print("loop exited")
            if (not self.should_quit):
                time.sleep(1)
        print("thread exit")    
        self.running = False

In [98]:
bufsize = 4096
buf = numpy.zeros(bufsize,dtype=numpy.float32)
mx = numpy.ones(bufsize,dtype=numpy.float32)*1.0
mn = numpy.ones(bufsize,dtype=numpy.float32)*-1.0 #1.0

sampleNumber = 0
sine_freqs = numpy.arange(220,700,100)
sine_pos = numpy.zeros(sine_freqs.size)
REAL_TIME_FREQUENCY = 220.0
ANGULAR_FREQUENCY = REAL_TIME_FREQUENCY * 2.0 * numpy.pi

def setFreq(hz):
    global REAL_TIME_FREQUENCY
    global ANGULAR_FREQUENCY
    REAL_TIME_FREQUENCY = hz
    ANGULAR_FREQUENCY = REAL_TIME_FREQUENCY * 2.0 * numpy.pi
    
interact(setFreq, hz=widgets.FloatSlider(min=20,max=20000,step=1,value=220))

def fillBuffer():
    global sampleNumber
    global sine_freqs
    global buf
    for i in range(0,bufsize):
        buf[i]=0
        components = sine_freqs.size
        sine_freqs = sine_freqs + numpy.random.uniform(-0.1,0.1,components)
        for j in range(0,components):
            sine_pos[j] += (sine_freqs[j]*2.0 * numpy.pi)/44100.0
            buf[i] += numpy.sin(sine_pos[j])/components
        
    tmp = numpy.fft.fft(buf)
    tmp[20:tmp.size-20]=0 #,dtype=numpy.complex)
    tmp = numpy.fft.ifft(tmp)
    buf = numpy.real(tmp).astype(numpy.float32)
    buf = numpy.minimum(buf,mx)
    buf = numpy.maximum(buf,mn)     
        #sampleTime = sampleTime + 1 / 44100.0 #sampleNumber / 44100.0
        #sampleAngle = sampleTime * ANGULAR_FREQUENCY
        #buf[i] = numpy.sin(sampleAngle)
        #sampleNumber += 1
print("components", sine_freqs.size)
fillBuffer()



print(buf[0:10])

#tmp = numpy.fft.fft(buf)
#tmp = numpy.fft.ifft(tmp)
#print(numpy.real(tmp[0:10]))


components 5
[ 0.35551405  0.35626394  0.35682222  0.35718837  0.35736197  0.35734275
  0.35713059  0.35672551  0.35612771  0.3553375 ]


In [4]:
server = None
loop_thread = None

In [99]:
class SignalGeneratorProtocol(WebSocketServerProtocol):
    def onConnect(self, request):
        print("Client connecting2: {}".format(request.peer))

    def onOpen(self):
        print("WebSocket connection open2.")

    def onMessage(self, payload, isBinary):
        #if isBinary:
        #    print("Binary message received: {} bytes".format(len(payload)))
        #else:
        #    print("Text message received: {}".format(payload.decode('utf8')))

        # echo back message verbatim
        #self.sendMessage(payload, isBinary)
        #print("fillb")
        fillBuffer()
        #print("send1")
        self.sendMessage(buf.tobytes(), True)
        #print("send2")
           
    def onClose(self, wasClean, code, reason):
        print("WebSocket connection closed: {}".format(reason))

In [100]:
factory = WebSocketServerFactory()
factory.protocol = SignalGeneratorProtocol

In [101]:
if server is not None:
    server.close()

if (loop_thread is not None):
    loop_thread.quit()
    loop_thread.join()
    
loop = asyncio.get_event_loop()
coro = loop.create_server(factory, '0.0.0.0', 8889)
server = loop.run_until_complete(coro)

loop_thread = WebIOThread()
loop_thread.start()

loop exited
thread exit
thread start
start loop


In [ ]:
async def do_hello():
    async with websockets.connect('ws://localhost:8889') as websocket:

        name = "dinne"
        await websocket.send(name)
        print("> v {}".format(name))

        greeting = await websocket.recv()
        print("< w {}".format(greeting))

#asyncio.get_event_loop().run_until_complete(do_hello())

In [ ]:
%%javascript
window.audioContext = new AudioContext();

In [ ]:
%%javascript
const REAL_TIME_FREQUENCY = 220;//440;
const ANGULAR_FREQUENCY = REAL_TIME_FREQUENCY * 2 * Math.PI;


var bufferSize = 4096;
window.brownNoise = (function() {
    var lastOut = 0.0;
    var sampleNumber = 0;
    var node = window.audioContext.createScriptProcessor(bufferSize, 1, 1);
    node.onaudioprocess = function(e) {
        var output = e.outputBuffer.getChannelData(0);
        for (var i = 0; i < bufferSize; i++) {
            var white = Math.random() * 2 - 1;
            output[i] = (lastOut + (0.02 * white)) / 1.02;
            lastOut = output[i];
            output[i] *= 3.5; // (roughly) compensate for gain
            
              let sampleTime = sampleNumber / 44100;
              let sampleAngle = sampleTime * ANGULAR_FREQUENCY;
              output[i] =  Math.sin(sampleAngle);
              sampleNumber += 1;
        }
    }
    return node;
})();




In [ ]:
%%javascript
brownNoise.connect(window.audioContext.destination);
setTimeout(function(){
    brownNoise.disconnect();
//    window.alert("done");
},6000);

//setTimeout(function(){ alert("Hello"); }, 3000);

In [ ]:
def generator():
    return [1,2,3]

In [ ]:
%%javascript

var kernel = IPython.notebook.kernel;

var callbacks = {
        shell : {
            reply: function(out) {
                var res = out.content.user_expressions.output;
                console.log(res);
                var data = res.data["text/plain"]; 
                console.log(data);
                //output = out.content.user_expressions.output;
                //res=output.data;
                //console.log(res);
            }
        },
        iopub : {
             output : function() {
                 console.log("more");
             }
    }
}

function handle_output3(x){
    console.log("zooi2");
    //window.alert(out_type+" blaat "+out_data);
}
kernel.execute("", callbacks,{user_expressions:{output: "generator()"}});

In [ ]:
%%javascript
console.log("zooi");


In [102]:
%%javascript 

if (!window.audioContext) {
    window.audioContext = new AudioContext();
}
// Let us open a web socket
var ws = new WebSocket("ws://127.0.0.1:8889");
ws.binaryType = 'arraybuffer';
window.ws=ws;

const REAL_TIME_FREQUENCY = 220;//440;
const ANGULAR_FREQUENCY = REAL_TIME_FREQUENCY * 2 * Math.PI;

var bufferSize = 4096;
var player = (function() {
    var sampleNumber = 0;
    var node = window.audioContext.createScriptProcessor(bufferSize, 1, 1);
    node.onaudioprocess = function(e) {
        var output = e.outputBuffer.getChannelData(0);
        if (ws.received_buffers.length>=1) {
            output.set(ws.received_buffers.shift());
        }
        /**for (var i = 0; i < bufferSize; i++) {
              output[i] = ws.last_received_buffer[i];  
              let sampleTime = sampleNumber / 44100;
              let sampleAngle = sampleTime * ANGULAR_FREQUENCY;
              output[i] =  Math.sin(sampleAngle);
              sampleNumber += 1;
        }**/
        // Web Socket is connected, send data using send()
        if (ws.received_buffers.length<10) {
            if (ws.readyState==1) {
                ws.send("Get block");
                ws.send("Get block");
            }
        }
    }
    return node;
})();


				
               ws.onopen = function()
               {
                  //console.log("Message is sent...");
                  ws.received_buffers = [] ;
                  ws.received_frames = 0
                  ws.send("Get block");
                  ws.is_connected = false;
                                   
               };
				
               ws.onmessage = function (evt) 
               { 
                  ws.received_buffers.push(new Float32Array(evt.data)); //.set(evt.data);
                  //ws.last_received_buffer.set(evt.data);
                  ws.received_frames+=1;
                  if (!ws.is_connected) {
                      player.connect(window.audioContext.destination);  
                      ws.is_connected = true;
                  }
                   //var received_msg = evt.data;
                  //console.log("Message is received...");
                  //console.log(ws.last_received_buffer);
                 
                  /** 
                  var wordarray = new Float32Array(evt.data);
                  for (var i = 0; i < 10; i++) //wordarray.length; i++) 
                  {
                    console.log(wordarray[i]);
                  }**/
               };
				
               ws.onclose = function()
               { 
                  player.disconnect();

                  // websocket is closed.
                  console.log("Connection is closed..."); 
               };
            

<IPython.core.display.Javascript object>

Client connecting2: tcp:172.18.0.1:36908
WebSocket connection open2.


In [103]:
%%javascript 
console.log(window.ws.received_buffers.length)
window.ws.close()

<IPython.core.display.Javascript object>

WebSocket connection closed: None
